In [1]:
import pandas as pd
import matplotlib.pyplot as plt

#df = pd.read_csv("localtime.csv", index_col=[0])
df = pd.read_csv("DF.csv", )

df = df.drop(['date', 'Connection','Accept-Encoding','CountryName'], 1)

#df = df.drop(['Accept','date', 'Connection','Accept-Encoding', 'Accept-Language','CountryName','City'], 1)
#df

In [2]:
df['path']=df['path'].str[:1]
#df = df.groupby(["IP","cookie","path"]).size().to_frame()


In [3]:
df2 = df.groupby(["IP","cookie","User-Agent","CountryCode","path","Accept-Language"]).size().reset_index(name="count")
df2 = df2
df2.head()

,IP,cookie,User-Agent,CountryCode,path,Accept-Language,count
0,101.242.160.152,"02/14/2020, 22:55:33",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,p,lu,4
1,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,k,lu,2
2,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,p,lu,2
3,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,k,lu,2
4,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,p,lu,1


In [4]:
df_p = df2[df2['path']=="p"].rename(columns={"count": "p"}).drop(["path"],1)
df_k = df2[df2['path']=="k"].rename(columns={"count": "k"}).drop(["path"],1)

In [5]:
df_m = df_p.merge(df_k, on=['cookie','IP', 'User-Agent', 'CountryCode','Accept-Language'], how='outer')
df_m.head()

,IP,cookie,User-Agent,CountryCode,Accept-Language,p,k
0,101.242.160.152,"02/14/2020, 22:55:33",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,4.0,NaN
1,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,2.0,2.0
2,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,1.0,2.0
3,101.242.160.152,"02/14/2020, 22:57:00",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,2.0,5.0
4,101.242.160.152,"02/14/2020, 23:21:16",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,4.0,2.0


In [6]:
def select_favorite(p,k):
    if pd.isna(p): 
        return "Kitten"
    elif pd.isna(k):
        return "Puppy"
    elif (p==k):
        return "Same"
    elif (p>k):
        return "Puppy"
    else:
        return "Kitten"
    

In [7]:
df_m['favorite'] = df_m.apply(lambda row: select_favorite(row['p'],row['k']), axis=1)
df_m.head()


,IP,cookie,User-Agent,CountryCode,Accept-Language,p,k,favorite
0,101.242.160.152,"02/14/2020, 22:55:33",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,4.0,NaN,Puppy
1,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,2.0,2.0,Same
2,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,1.0,2.0,Kitten
3,101.242.160.152,"02/14/2020, 22:57:00",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,2.0,5.0,Kitten
4,101.242.160.152,"02/14/2020, 23:21:16",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,4.0,2.0,Puppy


In [8]:
df_m2 = df_m.drop(['IP', 'cookie','p', 'k'], 1)
df_m2.head()


,User-Agent,CountryCode,Accept-Language,favorite
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,Puppy
1,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,Same
2,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,Kitten
3,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,Kitten
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,Puppy


In [9]:
import pycountry_convert as pc

def country_to_cont(cc):
    cont_code = pc.country_alpha2_to_continent_code(cc)
    cont_name = pc.convert_continent_code_to_continent_name(cont_code)
    return cont_name





In [10]:
df_t = df_m2.loc[df_m2['CountryCode'].notna()].copy(deep=True)
df_t.head()
df_t['Continent'] = df_t.apply(lambda row: country_to_cont(row['CountryCode']), axis=1)
df_t = df_t.drop(['CountryCode'],1)
df_t.to_csv("favorites.csv",index=False)
#df_t['localtime']=df_t['localtime'].str[:3]

In [16]:
df_t['IP']="*"
df_t['cookie']="*"
#df_t['Accept-Language']="*"
df_t['city']="*"
df_t.head(10)

,User-Agent,Accept-Language,favorite,Continent,IP,cookie,city
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Puppy,Asia,*,*,*
1,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Same,Asia,*,*,*
2,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Kitten,Asia,*,*,*
3,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Kitten,Asia,*,*,*
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Puppy,Asia,*,*,*
5,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Same,Asia,*,*,*
6,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Puppy,Asia,*,*,*
7,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Puppy,Asia,*,*,*
8,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Puppy,Asia,*,*,*
9,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,lu,Puppy,Asia,*,*,*


In [12]:
# from pytz import country_timezones 
# df_c = df_t 
# df_c['datetime'] = df_c.apply(lambda x: country_timezones(x['CountryCode'])[0], axis=1)

In [15]:
df_t.groupby(['Continent','User-Agent','Accept-Language']).size().reset_index().sort_values(by=0)
#dups = df_t.pivot_table(index=['Continent','User-Agent','Accept-Language'], aggfunc='size')

,Continent,User-Agent,Accept-Language,0
35,Asia,Mozilla/5.0 (X11; Linux i686; rv:10.0) Gecko/2...,lt,120
20,Asia,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5...,z,142
156,South America,Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-...,s,144
74,Europe,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,os,146
28,Asia,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:10...,jv,150
105,North America,Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-...,s,157
21,Asia,Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-...,s,159
86,North America,Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AO...,ko,160
40,Asia,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,os,161
48,Europe,Mozilla/2.0 (compatible; MSIE 4.0; Windows 98),s,163


In [14]:
dups.sort_values(0).to_frame()

NameError: name 'dups' is not defined

In [ ]:
from pytz import country_timezones 
df_c = df_t 
df_c['datetime'] = df_c.apply(lambda x: x['datetime'].tz_convert(country_timezones(x['CountryCode'])[0]), axis=1)